## Imports

In [8]:
#export
from torchvision import models
from torch import nn

## Get

In [7]:
#export
def get_model(god):
    model_name = god.config['model']['name']
    clf_type = god.config['model']['classification_type']
    
    if model_name == 'resnet18':
        model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        model.fc = get_final_layer(god, clf_type, model, model.fc.in_features)
    
    elif model_name == 'resnet152':
        model = models.resnet152(weights=models.ResNet152_Weights.IMAGENET1K_V2)
        model.fc = get_final_layer(god, clf_type, model, model.fc.in_features)
        
    elif model_name == 'vit_l_16':
        model = models.vit_l_16(weights=models.ViT_L_16_Weights.IMAGENET1K_SWAG_E2E_V1)
        model.heads = get_final_layer(god, clf_type, model, model.heads[0].in_features)
    
    elif model_name == 'convnext_large':
        model = models.convnext_large(weights=models.ConvNeXt_Large_Weights)
        raise Exception('TODO: Allow final layer to be added')
        
    elif model:
        raise Exception(f"Model with name '{model_name}' not supported yet")
        
    model.name = f"{model_name}_{clf_type}_{god.config['dataset']['path'].split('/')[-1]}"
    return model


def get_final_layer(god, clf_type, model, in_features):
    if clf_type == 'multiclass':
        return nn.Linear(in_features, len(god.config['id2label']))

    elif clf_type == 'binary':
        return nn.Sequential(nn.Linear(in_features, 1), nn.Sigmoid())

    elif clf_type == 'custom':
        return nn.Linear(in_features, god.config['model']['n_outputs'])
        
    elif clf_type == 'multiclass_sigmoid':
        return nn.Sequential(
            nn.Linear(in_features, len(god.config['id2label'])),
            nn.Sigmoid()
        )

    elif clf_type == 'multiclass_threshold':
        return nn.Sequential(
            nn.Linear(in_features, len(god.config['id2label']) - 1),
            nn.Sigmoid()
        )

    else: raise Exception(f"Classification type '{clf_type}' not supported yet")
        
    # TODO add support for this hierarchial loss
    # elif model_name == 'resnet18' and clf_type == 'fish_hierarchical':
    #     model = HierarchicalResNet18(num_classes=len(god.config['id2label']) - 1)    

# Models

In [3]:
#export
class HierarchicalResNet18(nn.Module):
    
    def __init__(self, num_classes):
        super(HierarchicalResNet18, self).__init__()
        self.base_model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)  # Use pre-trained weights
        self.fish_not_fish_classifier = nn.Linear(self.base_model.fc.in_features, 2)
        self.fish_type_classifier = nn.Linear(self.base_model.fc.in_features, num_classes)
        self.base_model.fc = nn.Identity()  # Keep everything up to the final layer

    def forward(self, x):
        x = self.base_model(x)  # Use pre-trained model's forward, which now stops at the penultimate layer

        fish_not_fish_output = self.fish_not_fish_classifier(x)
        fish_type_output = self.fish_type_classifier(x)

        if self.training:
            return fish_not_fish_output, fish_type_output
        else:
            return fish_not_fish_output, fish_type_output

## Test+Export

In [1]:
!python _notebook2script.py 03_Model.ipynb

Converted 03_Model.ipynb to exp/Model.py
